라이브러리 임포트

In [ ]:
import cv2
import tensorflow.lite as tflite
import numpy as np
import matplotlib.pyplot as plt
import base64
from sklearn.cluster import KMeans
from flask import Flask, request, jsonify, render_template
from PIL import Image

모델 및 클래스 설정
- 두피 상태를 분류하기 위한 클래스 목록
- 모발 개수를 탐지하기 위한 클래스 목록

In [ ]:
app = Flask(__name__)

# 클래스 이름 설정
classification_names = ['normal', 'mild', 'moderate', 'severe']
detection_names = ['1hair', '2hair', '3hair', '4hair']

# 모델 로드
classification_model_path = 'hlcM_torchQ16.tflite'
detection_model_path = 'yolo300_float32.tflite'

classification_interpreter = tflite.Interpreter(model_path=classification_model_path)
classification_interpreter.allocate_tensors()
class_input_details = classification_interpreter.get_input_details()
class_output_details = classification_interpreter.get_output_details()

detection_interpreter = tflite.Interpreter(model_path=detection_model_path)
detection_interpreter.allocate_tensors()
detect_input_details = detection_interpreter.get_input_details()
detect_output_details = detection_interpreter.get_output_details()

# YOLO 모델의 입력 이미지 크기 shape에서 추출
detect_img_height = detect_input_details[0]['shape'][1]
detect_img_width = detect_input_details[0]['shape'][2]


이미지 처리 함수 process_image
- 분류 모델 입력에 맞게 이미지 전처리
- TF-Lite 분류 모델 추론 수행
- 예측 결과를 통해 두피 상태 라벨 반환

In [ ]:
def process_image(image):
    if not isinstance(image, np.ndarray):
        print(f"오류: 지원되지 않는 이미지 형식 ({type(image)})")
        return None

    #BGR 이미지 RGB로 변환
    original_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = original_image.shape

    # 두피 상태 분류
    class_image = cv2.resize(image, (224, 224))
    class_image = np.array(class_image, dtype=np.float32)
    class_image = np.expand_dims(class_image, axis=0)

    classification_interpreter.set_tensor(class_input_details[0]['index'], class_image)
    classification_interpreter.invoke()
    class_output = classification_interpreter.get_tensor(class_output_details[0]['index'])
    class_pred = np.argmax(class_output[0])
    class_label = classification_names[class_pred]

- YOLO 모델을 사용해 입력 이미지에서 모발 개수 탐지
- 출력 처리
  - boxes_xywh : 중심 좌표 기준 박스 정보
  - score, classes : 클래스별 confidence 추출
- 최종 탐지된 결과 이미지에 바운딩 박스 및 라벨 시각화

In [ ]:
    # 이미지 전처리
    detect_image = cv2.resize(image, (detect_img_width, detect_img_height))
    detect_image = np.array(detect_image, dtype=np.float32) / 255.0
    detect_image = np.expand_dims(detect_image, axis=0)

    # YOLO 모델
    detection_interpreter.set_tensor(detect_input_details[0]['index'], detect_image)
    detection_interpreter.invoke()
    output = detection_interpreter.get_tensor(detect_output_details[0]['index'])[0].T

    if output.shape[1] == 8:
        boxes_xywh = output[:, :4]
        scores = np.max(output[:, 4:], axis=1)
        classes = np.argmax(output[:, 4:], axis=1)
    else:
        print("오류: YOLO 모델 출력 형식 확인 필요.")
        return None

    threshold = 0.2
    iou_threshold = 0.4

    boxes, confidences, class_ids = [], [], []
    for i, s in enumerate(scores):
        if s > threshold:
            x_center, y_center, width, height = boxes_xywh[i] * [img_w, img_h, img_w, img_h]
            x1, y1, x2, y2 = int(x_center - width / 2), int(y_center - height / 2), int(x_center + width / 2), int(
                y_center + height / 2)
            boxes.append([x1, y1, x2, y2])
            confidences.append(float(s))
            class_ids.append(classes[i])

    # 이미지에 결과 시각화 표시
    indices = cv2.dnn.NMSBoxes(boxes, confidences, threshold, iou_threshold)
    if len(indices) > 0:
        indices = indices.flatten()
        for i in indices:
            x1, y1, x2, y2 = boxes[i]
            cls = class_ids[i]
            score = confidences[i]
            cv2.rectangle(original_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            text = f"{detection_names[cls]} ({score:.2f})"
            cv2.putText(original_image, text, (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

KMeans로 두피/모발 비율 산출
- 입력 이미지 LAB 컬러 공간으로 변환 후 2개의 군집으로 수행
- 두피(흰색), 머리카락(검정색)으로 픽셀값 이진화
- 머리카락과 두피 픽셀 수 기반으로 각각의 비율 계산

In [ ]:
    # KMeans 클러스터링
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    pixels = lab_image.reshape((-1, 3))

    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = kmeans.fit_predict(pixels)
    segmented_image = labels.reshape(image.shape[:2])

    #  두피 : 흰색(255), 머리카락 : 검정색(0)
    if np.mean(segmented_image) > 0.5:
        segmented_image = 1 - segmented_image

    segmented_image = (segmented_image * 255).astype(np.uint8)

    # hair_ratio, scalp_ratio 계산
    hair_pixels = np.sum(segmented_image == 0)
    scalp_pixels = np.sum(segmented_image == 255)
    total_pixels = hair_pixels + scalp_pixels

    hair_ratio = (hair_pixels / total_pixels) * 100 if total_pixels > 0 else 0
    scalp_ratio = (scalp_pixels / total_pixels) * 100 if total_pixels > 0 else 0



Flask에서 표시하기 위해 base64로 인코딩 후 JSON 응답으로 변환

In [ ]:
 _, buffer1 = cv2.imencode(".jpg", cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR))
    image_base64_1 = base64.b64encode(buffer1).decode("utf-8")

    _, buffer2 = cv2.imencode(".jpg", cv2.cvtColor(segmented_image, cv2.COLOR_GRAY2RGB))
    image_base64_2 = base64.b64encode(buffer2).decode("utf-8")

    result = {
        "classification": class_label,
        "detection": [
            {"box": boxes[i], "confidence": confidences[i], "label": detection_names[class_ids[i]]} for i in indices
        ],
        "segmentation": {
        "hair_ratio": round(hair_ratio, 2),
        "scalp_ratio": round(scalp_ratio, 2)},
        "image1": image_base64_1,  # 탐지 결과
        "image2": image_base64_2   # 클러스터링 결과
    }
    return result

Flask 엔드포인트
- upload.html에서 이미지 업로드
- 서버에서 이미지 분석
- 분석 결과를 result.html에 전달하여 시각화

에러
- 이미지가 없는 경우 : 400
- 분석 실패한 경우 : 500

In [ ]:
# Flask 엔드포인트
@app.route('/', methods=['GET', 'POST'])

# 업로드 및 결과 페이지 렌더링
def home():
    return render_template("upload.html")

@app.route("/result")
def scalp_record():
    return render_template("result.html")

@app.route('/file', methods=['POST'])
def analyze():
    if 'image' not in request.files:
        return jsonify({"error": "No image uploaded"}), 400

    file = request.files['image']
    image = Image.open(file.stream).convert('RGB')
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    result = process_image(image)

    if result is None:
        return jsonify({"error": "Failed to process image"}), 500

    return render_template("result.html", uploaded_image1=f"data:image/jpeg;base64,{result['image1']}",
                           uploaded_image2=f"data:image/jpeg;base64,{result['image2']}",
                           result=result)


# 서버 실행
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8156)

